In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch import optim

In [3]:
base_dir = os.path.dirname(os.getcwd())
sys.path.append(base_dir)

from core.dataloader import CelebALoader, PokeLoader
from core.models import VAE, Discriminator, Generator
from core.models import modules
from core.engine import ConfigFile, NCTrainer
from core.metrics import *
data_dir = "../../tmp"

# ./Session

In [3]:
config = ConfigFile("")

In [4]:
config.set_session_name("sandbox_mix")
config.setup_session()

/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/chkpt already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/runs already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/scores already exists
  warnings.warn(f"directory {full_path} already exists")



# ./Dataloader

In [30]:
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                                     std=(0.5, 0.5, 0.5))])

dataloader = PokeLoader(data_dir=os.path.join(data_dir, 'pokemon'), 
                          batch_size=32,
                          train_transform=transform,
                          val_transform=transform,
                          validation_split=0.05)
config.set_dataloader(dataloader)

In [5]:
transform = transforms.Compose([transforms.Resize((64, 64)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                                     std=(0.5, 0.5, 0.5))])

sampler = torch.utils.data.SubsetRandomSampler(indices=np.random.choice(96000, size=96000))

dataloader = CelebALoader(data_dir=data_dir, 
                          batch_size=128,
                          train_transform=transform,
                          val_transform=transform,
                          sampler=sampler,
                          validation_split=0.3)
config.set_dataloader(dataloader)

__Mask Generator :__

In [6]:
masks_kwargs = {'size': (64, 64),
                'coverage': (0.1, 0.5)}

config.update_kwargs(masks=masks_kwargs)

# ./Model

__Generator :__

In [6]:
kwargs =  [{'stride': 1, 'padding': 0}] + 3 * [{}] + [{'kernel_size': 4, 'bn': False, 'relu': False}]

generator = Generator(latent_size=(100, 1, 1),
                      nb_filters=[512, 256, 128, 64, 3],
                      conv_kwargs=kwargs)
config.set_model(generator)

__VAE :__

In [9]:
# vae = VAE(input_size=(6, 64, 64), 
#           z_dim=100, 
#           enc_nf = [64, 128, 128], 
#           dec_nf = [256, 128, 128],
#           enc_kwargs = {'padding': 1},
#           dec_kwargs = {'kernel_size': 4},
#           out_kwargs = {'kernel_size': 4, 'relu': False, 'bn': False},
#           out_channels=3)
# config.set_model(vae)

__Discriminator :__

In [4]:
kwargs = [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': False}] + \
         3 * [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}]

disc_kwargs = {'input_size': (3, 64, 64),
              'nb_filters': [512, 256, 128, 128],
              'conv_kwargs': kwargs}
# config.update_kwargs(discriminator=disc_kwargs)

# ./Training params

__Criterion :__

In [8]:
criterion = nn.BCELoss()
config.set_criterion(criterion)

__Optimizers:__

In [9]:
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
disc_optimizer = {'lr': 2e-4,
                  'betas': (0.5, 0.999)}

config.set_optimizer(gen_optimizer)
config.update_kwargs(disc_optimizer=disc_optimizer)

__LR Scheduler:__

In [34]:
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=config.optimizer, gamma=0.999)

config.set_lr_scheduler(scheduler)

__Metrics:__

In [16]:
callables = [accuracy, precision, recall]
config.set_metrics(callables)

__Mixup:__

In [11]:
mixup_kwargs = {'alpha': 0.75}
config.update_kwargs(mixup=mixup_kwargs)

__Training scope :__

In [12]:
epoch = 128
config.set_epochs(epoch)

In [17]:
config.dump()

In [14]:
config

_session_name : sandbox_mix
_session_dir : /home/shahine/neural-conditioner/bin/sandbox_mix
_criterion : BCELoss()
_optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)
_metrics : None
_dataloader : Dataset CelebA
    Number of datapoints: 162770
    Root location: ../../tmp
    Target type: ['attr']
    Split: train
_init_epoch : 0
_epochs : 128
_lr_scheduler : None
_seed : 73
_kwargs : {'discriminator': {'input_size': (3, 64, 64), 'nb_filters': [512, 256, 128, 128], 'conv_kwargs': [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': False}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}]}, 'disc_optimizer': {'lr': 0.0002, 'betas': (0.5, 0.999)}, 'mixup': {'alpha': 0.75}}

In [46]:
!tree -d ../bin

../bin
├── sandbox_gans
│   ├── chkpt
│   ├── runs
│   └── scores
└── sandbox_gans_2
    ├── chkpt
    ├── runs
    └── scores

8 directories


$$\log IS(G) = \mathbb{E}_{x\sim p_G} KL\left[p(y|x)\|p(y)\right]\sim \frac{1}{K}\sum_{k=1}^{K}\frac{1}{n_k} \sum_{i=1}^{n_k} KL\left[p(y_k^{(i)}|x_k^{(i)})\|p(\hat y_k)\right]$$

In [32]:
import random
import torch
import torch.nn.functional as F

def inception_score(fake_samples, inception_model, split_size=4):
    """Salimans et al. (2016)
    Args:
        fake_samples (torch.Tensor): batch of fake generated images
        inception_model (nn.Module): inception model
        split_size (int): number of samples to consider for marginal computation
    """
    with torch.no_grad():
        pred = torch.softmax(inception_model(fake_samples).logits, dim=-1)
    conditionals = torch.stack(pred.split(split_size))
    marginals = conditionals.mean(dim=1, keepdim=True).repeat(1, split_size, 1)
    kl = F.kl_div(conditionals, marginals, reduction='batchmean')
    return torch.exp(kl).item()

In [24]:
from torchvision.models.inception import inception_v3
inception_model = inception_v3(pretrained=True, transform_input=False).to(torch.device('cuda:2'))

In [33]:
torch.random.manual_seed(55)
fake_samples = 0.5 + 0.5 * torch.randn((128, 3, 300, 300), device=torch.device('cuda:2'))
kl = inception_score(fake_samples, inception_model)

In [47]:
conditionals = torch.stack(64 * [torch.eye(10)])
conditionals = conditionals + torch.rand_like(conditionals)
marginals = conditionals.mean(dim=1, keepdim=True).repeat(1, 10, 1)

In [61]:
kl = F.kl_div(conditionals[0, 0], marginals[0,0])

In [62]:
kl

tensor(-0.7630)

In [63]:
torch.sum(conditionals[0, 0] * torch.log(conditionals[0, 0] / marginals[0, 0]))

tensor(2.5348)

In [34]:
kl

3.0085814482078144e-12

In [52]:
marginals[0, 0]

tensor([0.5853, 0.6333, 0.5924, 0.6032, 0.6558, 0.6284, 0.5099, 0.7034, 0.8330,
        0.6891])

In [42]:
F.kl_div(conditionals[0, 0], marginals[0, 0])

tensor(-0.2403)

In [86]:
kl_ = F.kl_div(conditionals.view_as(pred), marginals.view_as(pred))

/home/shahine/tmp/.venv/lib/python3.6/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


In [92]:
torch.exp(kl.mean()).item()

1.0379011631011963

In [75]:
kl = entropy(conditionals.view_as(pred).t().numpy(), marginals.view_as(pred).t().numpy())
kl = torch.Tensor(kl)

In [80]:
torch.stack(kl.split(4)).mean(dim=0).mean()

tensor(0.0372)

In [81]:
kl.mean()

tensor(0.0372)

In [71]:
kl[0]

0.034630544

In [66]:
conditionals[0, 0].size()

torch.Size([10])